In [37]:
import os

In [38]:
%pwd

'd:\\'

In [39]:
os.chdir("../")

In [40]:
%pwd

'd:\\'

In [41]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [42]:
from src.textSummarizer.constants import *


In [43]:
pip install python-box==5.4.1

Note: you may need to restart the kernel to use updated packages.


In [44]:
from src.textSummarizer.utils.common import read_yaml, create_directories

In [45]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
    
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
    
        return data_ingestion_config

In [46]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [47]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        

In [48]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-09 18:14:48,012: INFO: common: yaml file: D:\Text_summarizer_project\config\config.yaml loaded successfully]
[2025-03-09 18:14:48,016: INFO: common: yaml file: D:\Text_summarizer_project\params.yaml loaded successfully]
[2025-03-09 18:14:48,019: INFO: common: created directory at: artifacts]
[2025-03-09 18:14:48,021: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-09 18:14:50,580: INFO: 2799438964: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 46E2:361665:411A19:511715:67CE12D9
Accept-Ranges: bytes
Date: Sun, 09 Mar 2025 22:14:49 GMT
Via: